In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Data Cleaning/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Data Cleaning/Output')
cur_date = '030923'

library(readxl)
library(tidyverse)

# reading in files
county_df = data.frame(read_excel("nc.xls"))
whp_df = data.frame(read_excel("Wildfire_Hazard_010323.xlsx", sheet = 2))
# making the last 2 cols numeric
whp_df[,3:4] = apply(whp_df[,3:4], 2, as.numeric)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message in apply(whp_df[, 3:4], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(whp_df[, 3:4], 2, as.numeric):
“NAs introduced by coercion”


In [2]:
head(county_df)
head(whp_df)

,STATE,LOGRECNO,GEO_ID,GEOGRAPHY.NAME
,<chr>,<chr>,<chr>,<chr>
1,NC,0000001,04000US37,North Carolina
2,NC,0000002,04001US37,North Carolina -- Urban
3,NC,0000003,04043US37,North Carolina -- Rural
4,NC,0000004,040A0US37,North Carolina -- In metropolitan or micropolitan statistical area
5,NC,0000005,040C0US37,North Carolina -- In metropolitan statistical area
6,NC,0000006,040C1US37,North Carolina -- In metropolitan statistical area -- in principal city


,GEO_ID,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37119005916,59.16,349323.3,139.2436
2,14000US37119003808,38.08,349323.3,139.2436
3,14000US37119005918,59.18,349323.3,139.2436
4,14000US37119005914,59.14,349323.3,139.2436
5,14000US37071031004,310.04,233025.3,188.1261
6,14000US37071031201,312.01,291174.3,163.6849


In [3]:
length(unique(whp_df$GEO_ID))

[1] 2194

In [4]:
whp_df$GEO_ID[duplicated(whp_df$GEO_ID)]

[1] "14000US37101041103"

NC has 2,195 geo ids/ census tracts, so we're missing one.

In [5]:
# figuring out if we have any missing data in our whp df
whp_df[!complete.cases(whp_df),]

,GEO_ID,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<dbl>,<dbl>,<dbl>
372,14000US37095990100,9901,NA,NA
649,14000US37031990200,9902,NA,NA
819,14000US37095990200,9902,NA,NA
865,14000US37137990100,9901,NA,NA
1229,14000US37055990200,9902,NA,NA


These 5 census tracts are located on the coast and have a population of 0, which is probably why there's no data for them. However, I'll keep them in the dataset and impute their values.

In [6]:
# removing the duplicate entry
whp_df = unique(whp_df) %>%
# adding an entry to the whp_df for the missing geo id to make join easier
    add_row(GEO_ID = "14000US37101041202", Census_Tract = NA, Avg_ACRES = NA, 
            Wildfire_Hazard_Potential_Mean = NA) 

dim(whp_df)

[1] 2195    4

Great! Now we have 2,195 geo ids.

In [7]:
# adding in county names
combined_df = right_join(county_df[,3:4], whp_df) %>%
    # cleaning up county col
    separate(GEOGRAPHY.NAME, c(NA, "County", NA), sep = ", ")

head(combined_df)

Joining, by = "GEO_ID"


,GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,Alamance County,201.00,278233.8,92.23704
2,14000US37001020200,Alamance County,202.00,278233.8,92.23704
3,14000US37001020300,Alamance County,203.00,278233.8,92.23704
4,14000US37001020400,Alamance County,204.00,278233.8,92.23704
5,14000US37001020501,Alamance County,205.01,278233.8,92.23704
6,14000US37001020502,Alamance County,205.02,278233.8,92.23704


In [8]:
# NC has 2,195 census tracts (geo ids), how many do we have in this df?
length(unique(combined_df$GEO_ID))

[1] 2195

In [9]:
# exporting data
write.csv(combined_df, paste0(Output,"/", cur_date, "_WHP_Data.csv"), row.names = FALSE)